# Question 5

In [ ]:
!pip install python-igraph

In [ ]:
from math import *
import sys
import numpy as np
import os
import glob
import random
from random import shuffle
from random import seed
import matplotlib.pyplot as plt
import time
import datetime
import collections
import networkx as nx
from igraph import *

In [ ]:
k = 5

In [ ]:
import gzip
import shutil
with gzip.open('facebook_combined.txt.gz', 'rb') as f_in:
    with open('facebook_combined.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
data_file = open('facebook_combined.txt')
edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
data_file.close()

In [ ]:
def get_sim_product(sim, shortest_path):
	prod = 1
	for i in range(len(shortest_path) - 1):
		prod *= sim[shortest_path[i]][shortest_path[i+1]]
	return round(prod,3)

In [ ]:
# Filter out, Sort and Get top-K predictions
def get_top_k_recommendations(graph,sim,i,k):
	return  sorted(filter(lambda x: i!=x and graph[i,x] != 1,range(len(sim[i]))) , key=lambda x: sim[i][x],reverse=True)[0:k]

# Convert edge_list into a set of constituent edges
def get_vertices_set(edge_list):
	res = set()
	for x,y in edge_list:
		res.add(x)
		res.add(y)
	return res

In [ ]:
def split_data(edge_list):
  random.seed(350)
  edge_list = list(edge_list)
  indexes = range(len(edge_list))
  test_indexes = set (random.sample(indexes, int(len(indexes)/2)))
  train_indexes = set(indexes).difference(test_indexes)
  test_list = [edge_list[i] for i in test_indexes]
  train_list = [edge_list[i] for i in train_indexes]
  return train_list, test_list


In [ ]:
def print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set):
	precision = recall = c = 0
	for i in test_vertices_set:
		if i in train_vertices_set:
			actual_friends_of_i = set(test_graph.neighbors(i))


			if len(actual_friends_of_i) < k:
				k2 = len(actual_friends_of_i)
			else:
				k2 = k

			top_k = set(get_top_k_recommendations(train_graph,sim,i,k2))

			precision += len(top_k.intersection(actual_friends_of_i))/float(k2)
			recall += len(top_k.intersection(actual_friends_of_i))/float(len(actual_friends_of_i))
			c += 1
	print ("Precision is : " + str( (precision/c)))
	print ("Recall is : " + str((recall/c)))

In [ ]:
from scipy.sparse.csgraph import shortest_path

def similarity(graph, i, j, method):
	if method == "common_neighbors":
		return len(set(graph.neighbors(i)).intersection(set(graph.neighbors(j))))
	elif method == "jaccard":
		return len(set(graph.neighbors(i)).intersection(set(graph.neighbors(j))))/float(len(set(graph.neighbors(i)).union(set(graph.neighbors(j)))))
	elif method == "adamic_adar":
		return sum([1.0/math.log(graph.degree(v)) for v in set(graph.neighbors(i)).intersection(set(graph.neighbors(j)))])
	elif method == "shortest_path":
		adj = graph.get_adjacency_sparse()
		return shortest_path(adj)[i,j]
	else:
		raise Exception('Method not found!')

SyntaxError: ignored

In [ ]:
def local_methods(edge_list, method):
  train_list, test_list = split_data(edge_list)
  train_graph = Graph(train_list)
  test_graph = Graph(test_list)
  train_n =  train_graph.vcount()
  train_vertices_set = get_vertices_set(train_list)
  test_vertices_set = get_vertices_set(test_list)

  sim = [[0 for i in range(train_n)] for j in range(train_n)]
  for i in range(train_n):
    for j in range(train_n):
      if i!=j and i in train_vertices_set and j in train_vertices_set:
        sim[i][j] = similarity(train_graph,i,j,method)
  print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set)

In [ ]:
print("common_neighbors: ")
data_file = open('facebook_combined.txt')
edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
data_file.close()
local_methods(edge_list, method="common_neighbors")

common_neighbors: 
Precision is : 0.566554170993422
Recall is : 0.2014233086674809


In [ ]:
print("jaccard: ")
data_file = open('facebook_combined.txt')
edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
data_file.close()
local_methods(edge_list, method ="jaccard")

jaccard: 
Precision is : 0.4917358947732754
Recall is : 0.16007218514369487


In [ ]:
print("adamic_adar: ")
data_file = open('facebook_combined.txt')
edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
data_file.close()
local_methods(edge_list, method ="adamic_adar")

adamic_adar: 
Precision is : 0.592640186915887
Recall is : 0.217246398361926


In [ ]:
print("shortest_path: ")
data_file = open('facebook_combined.txt')
edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
data_file.close()
local_methods(edge_list, method="shortest_path")

shortest_path: 
